# 📈 Análise de Mercado Imobiliário com Gráficos Avançados

Este notebook demonstra:
- Análise comparativa de preços
- Tendências de mercado
- Gráficos de correlação
- Análise de performance de vendas
- Segmentação de mercado
- Dashboard de indicadores
- Design minimalista em preto e branco

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuração de estilo minimalista
plt.style.use('grayscale')
sns.set_palette("gray")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print("✓ Bibliotecas importadas com sucesso")

## 1. Carregar e Preparar Dados

In [ ]:
# Carregar dados de imóveis
try:
    df_imoveis = pd.read_parquet('../data/processed/imoveis_processados.parquet')
    print(f"✓ Dados carregados: {len(df_imoveis)} imóveis")
except FileNotFoundError:
    print("⚠️ Criando dados de exemplo...")
    
    np.random.seed(42)
    n = 300
    
    df_imoveis = pd.DataFrame({
        'Incorporador': np.random.choice(['Construtora Alpha', 'Construtora Beta', 'Construtora Gamma', 'Construtora Delta'], n),
        'Empreendimento': [f'Residencial {i}' for i in range(n)],
        'Bairro': np.random.choice(['Centro', 'Praia do Canto', 'Jardim Camburi', 'Enseada do Suá', 'Bento Ferreira'], n),
        'Endereco': [f'Rua {i%30}, {np.random.randint(1,500)}' for i in range(n)],
        'Cidade': ['Vitória'] * n,
        'Dormitorios': np.random.choice([1, 2, 3, 4], n, p=[0.15, 0.35, 0.35, 0.15]),
        'Metragem Privativa': np.random.uniform(35, 250, n),
        'Vagas': np.random.choice([0, 1, 2, 3], n, p=[0.1, 0.3, 0.4, 0.2]),
        'Preco Total': np.random.uniform(200000, 2500000, n),
        'Status': np.random.choice(['Lançamento', 'Em Construção', 'Pronto'], n, p=[0.3, 0.5, 0.2]),
        'Unidades Total': np.random.randint(20, 250, n),
        'Unidades Vendidas': np.random.randint(5, 200, n),
        'Estoque Atual': np.random.randint(5, 150, n)
    })
    
    # Ajustar correlações realistas
    df_imoveis['Preco Total'] = df_imoveis['Metragem Privativa'] * np.random.uniform(8000, 15000, n) + \
                                  df_imoveis['Dormitorios'] * np.random.uniform(50000, 150000, n)
    
    print(f"✓ Dados de exemplo criados: {len(df_imoveis)} imóveis")

# Calcular métricas derivadas
df_imoveis['Preco_m2'] = df_imoveis['Preco Total'] / df_imoveis['Metragem Privativa']
df_imoveis['Perc_Vendido'] = (df_imoveis['Unidades Vendidas'] / df_imoveis['Unidades Total']) * 100
df_imoveis['VGV'] = df_imoveis['Preco Total'] * df_imoveis['Unidades Total']
df_imoveis['VGV_Realizado'] = df_imoveis['Preco Total'] * df_imoveis['Unidades Vendidas']

print(f"\n✓ Métricas calculadas")
print(f"Colunas disponíveis: {list(df_imoveis.columns)}")

df_imoveis.head()

## 2. Dashboard de Indicadores Principais

In [ ]:
# Calcular indicadores
total_empreendimentos = len(df_imoveis)
preco_medio_m2 = df_imoveis['Preco_m2'].mean()
ticket_medio = df_imoveis['Preco Total'].mean()
perc_vendido_medio = df_imoveis['Perc_Vendido'].mean()
vgv_total = df_imoveis['VGV'].sum()
vgv_realizado = df_imoveis['VGV_Realizado'].sum()

# Criar figura de dashboard
fig = plt.figure(figsize=(16, 10))
fig.suptitle('📊 DASHBOARD - INDICADORES DO MERCADO IMOBILIÁRIO', fontsize=18, fontweight='bold', y=0.98)

# Grid layout
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# KPI Cards (primeira linha)
kpis = [
    ('Empreendimentos', f'{total_empreendimentos:,}', ''),
    ('Preço Médio/m²', f'R$ {preco_medio_m2:,.0f}', ''),
    ('Ticket Médio', f'R$ {ticket_medio/1000:,.0f}k', ''),
    ('% Vendido Médio', f'{perc_vendido_medio:.1f}%', ''),
    ('VGV Total', f'R$ {vgv_total/1e9:.2f}B', ''),
    ('VGV Realizado', f'R$ {vgv_realizado/1e9:.2f}B', '')
]

for i, (titulo, valor, subtitulo) in enumerate(kpis[:3]):
    ax = fig.add_subplot(gs[0, i])
    ax.text(0.5, 0.7, valor, ha='center', va='center', fontsize=28, fontweight='bold')
    ax.text(0.5, 0.3, titulo, ha='center', va='center', fontsize=12, color='gray')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    rect = Rectangle((0.05, 0.05), 0.9, 0.9, linewidth=2, edgecolor='black', facecolor='white')
    ax.add_patch(rect)

# Segunda linha de KPIs
for i, (titulo, valor, subtitulo) in enumerate(kpis[3:]):
    ax = fig.add_subplot(gs[1, i])
    ax.text(0.5, 0.7, valor, ha='center', va='center', fontsize=28, fontweight='bold')
    ax.text(0.5, 0.3, titulo, ha='center', va='center', fontsize=12, color='gray')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    rect = Rectangle((0.05, 0.05), 0.9, 0.9, linewidth=2, edgecolor='black', facecolor='white')
    ax.add_patch(rect)

# Gráfico de distribuição por status (terceira linha)
ax_status = fig.add_subplot(gs[2, :])
status_counts = df_imoveis['Status'].value_counts()
bars = ax_status.bar(status_counts.index, status_counts.values, 
                     color=['black', 'gray', 'lightgray'], 
                     alpha=0.7, edgecolor='black', linewidth=2)

# Adicionar valores nas barras
for bar in bars:
    height = bar.get_height()
    ax_status.text(bar.get_x() + bar.get_width()/2., height,
                   f'{int(height)}',
                   ha='center', va='bottom', fontweight='bold', fontsize=12)

ax_status.set_title('Distribuição de Empreendimentos por Status', fontsize=14, fontweight='bold', pad=10)
ax_status.set_xlabel('Status', fontsize=11)
ax_status.set_ylabel('Quantidade', fontsize=11)
ax_status.spines['top'].set_visible(False)
ax_status.spines['right'].set_visible(False)

plt.show()

print("✓ Dashboard de indicadores gerado")

## 3. Análise Comparativa de Preços por Bairro

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análise Comparativa de Preços por Bairro', fontsize=16, fontweight='bold', y=0.995)

# 1. Box Plot - Preço/m² por Bairro
bairros_ordenados = df_imoveis.groupby('Bairro')['Preco_m2'].median().sort_values(ascending=False).index
df_sorted = df_imoveis.set_index('Bairro').loc[bairros_ordenados].reset_index()

box_parts = axes[0, 0].boxplot([df_sorted[df_sorted['Bairro'] == b]['Preco_m2'].dropna() for b in bairros_ordenados],
                                labels=bairros_ordenados,
                                patch_artist=True,
                                showmeans=True)

for patch in box_parts['boxes']:
    patch.set_facecolor('lightgray')
    patch.set_edgecolor('black')
    patch.set_linewidth(1.5)

for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
    plt.setp(box_parts[element], color='black', linewidth=1.5)

axes[0, 0].set_title('Distribuição de Preço/m² por Bairro', fontweight='bold')
axes[0, 0].set_ylabel('Preço/m² (R$)')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3, axis='y')

# 2. Violin Plot - Preço Total por Bairro
parts = axes[0, 1].violinplot([df_sorted[df_sorted['Bairro'] == b]['Preco Total'].dropna() for b in bairros_ordenados],
                               positions=range(len(bairros_ordenados)),
                               showmeans=True,
                               showmedians=True)

for pc in parts['bodies']:
    pc.set_facecolor('lightgray')
    pc.set_edgecolor('black')
    pc.set_linewidth(1.5)
    pc.set_alpha(0.7)

axes[0, 1].set_xticks(range(len(bairros_ordenados)))
axes[0, 1].set_xticklabels(bairros_ordenados, rotation=45)
axes[0, 1].set_title('Distribuição de Preço Total por Bairro', fontweight='bold')
axes[0, 1].set_ylabel('Preço Total (R$)')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Barras - Comparação de Médias
preco_medio_bairro = df_imoveis.groupby('Bairro').agg({
    'Preco_m2': 'mean',
    'Preco Total': 'mean'
}).sort_values('Preco_m2', ascending=False)

x = np.arange(len(preco_medio_bairro))
width = 0.35

bars1 = axes[1, 0].bar(x - width/2, preco_medio_bairro['Preco_m2'], 
                       width, label='Preço/m²', color='black', alpha=0.7, edgecolor='white')

ax2 = axes[1, 0].twinx()
bars2 = ax2.bar(x + width/2, preco_medio_bairro['Preco Total']/1000, 
                width, label='Preço Total (mil)', color='gray', alpha=0.7, edgecolor='white')

axes[1, 0].set_xlabel('Bairro')
axes[1, 0].set_ylabel('Preço/m² (R$)', fontweight='bold')
ax2.set_ylabel('Preço Total (R$ mil)', fontweight='bold')
axes[1, 0].set_title('Comparação de Preço Médio/m² e Total', fontweight='bold')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(preco_medio_bairro.index, rotation=45)
axes[1, 0].legend(loc='upper left')
ax2.legend(loc='upper right')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# 4. Scatter - Preço/m² vs Quantidade de Imóveis
qtd_por_bairro = df_imoveis.groupby('Bairro').size()
preco_m2_bairro = df_imoveis.groupby('Bairro')['Preco_m2'].mean()

axes[1, 1].scatter(qtd_por_bairro, preco_m2_bairro, s=200, c='black', alpha=0.6, edgecolors='white', linewidth=2)

for bairro in qtd_por_bairro.index:
    axes[1, 1].annotate(bairro, 
                       (qtd_por_bairro[bairro], preco_m2_bairro[bairro]),
                       xytext=(5, 5), textcoords='offset points', fontsize=9)

axes[1, 1].set_xlabel('Quantidade de Imóveis')
axes[1, 1].set_ylabel('Preço Médio/m² (R$)')
axes[1, 1].set_title('Oferta vs Preço Médio por Bairro', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 RANKING DE PREÇOS POR BAIRRO")
print("="*60)
ranking = df_imoveis.groupby('Bairro').agg({
    'Preco_m2': 'mean',
    'Preco Total': 'mean',
    'Empreendimento': 'count'
}).round(2)
ranking.columns = ['Preço/m² Médio', 'Preço Total Médio', 'Qtd']
ranking = ranking.sort_values('Preço/m² Médio', ascending=False)
print(ranking)

## 4. Matriz de Correlação

In [ ]:
# Selecionar variáveis numéricas
numeric_cols = ['Dormitorios', 'Metragem Privativa', 'Vagas', 'Preco Total', 
                'Preco_m2', 'Unidades Total', 'Perc_Vendido']

# Calcular correlação
corr_matrix = df_imoveis[numeric_cols].corr()

# Criar figura
fig, ax = plt.subplots(1, 1, figsize=(12, 10))

# Heatmap em escala de cinza
im = ax.imshow(corr_matrix, cmap='Greys', aspect='auto', vmin=-1, vmax=1)

# Configurar ticks
ax.set_xticks(np.arange(len(numeric_cols)))
ax.set_yticks(np.arange(len(numeric_cols)))
ax.set_xticklabels(numeric_cols, rotation=45, ha='right')
ax.set_yticklabels(numeric_cols)

# Adicionar valores de correlação
for i in range(len(numeric_cols)):
    for j in range(len(numeric_cols)):
        text_color = 'white' if abs(corr_matrix.iloc[i, j]) > 0.5 else 'black'
        text = ax.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                      ha='center', va='center', color=text_color, fontweight='bold')

# Colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Correlação', rotation=270, labelpad=20, fontweight='bold')

ax.set_title('Matriz de Correlação - Variáveis do Mercado Imobiliário', 
             fontsize=14, fontweight='bold', pad=20)

# Grid
ax.set_xticks(np.arange(len(numeric_cols))-.5, minor=True)
ax.set_yticks(np.arange(len(numeric_cols))-.5, minor=True)
ax.grid(which='minor', color='black', linestyle='-', linewidth=2)

plt.tight_layout()
plt.show()

print("\n📈 CORRELAÇÕES MAIS FORTES:")
print("="*60)
# Extrair correlações mais fortes (exceto diagonal)
corr_pairs = []
for i in range(len(corr_matrix)):
    for j in range(i+1, len(corr_matrix)):
        corr_pairs.append((numeric_cols[i], numeric_cols[j], corr_matrix.iloc[i, j]))

corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)
for var1, var2, corr in corr_pairs[:5]:
    print(f"  • {var1:20s} ↔ {var2:20s}: {corr:+.3f}")

## 5. Análise de Performance de Vendas

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análise de Performance de Vendas', fontsize=16, fontweight='bold', y=0.995)

# 1. Distribuição de % Vendido
axes[0, 0].hist(df_imoveis['Perc_Vendido'], bins=30, color='black', alpha=0.7, edgecolor='white', linewidth=1.5)
axes[0, 0].axvline(df_imoveis['Perc_Vendido'].mean(), color='gray', linestyle='--', linewidth=2, label=f"Média: {df_imoveis['Perc_Vendido'].mean():.1f}%")
axes[0, 0].axvline(df_imoveis['Perc_Vendido'].median(), color='darkgray', linestyle=':', linewidth=2, label=f"Mediana: {df_imoveis['Perc_Vendido'].median():.1f}%")
axes[0, 0].set_title('Distribuição de % Vendido', fontweight='bold')
axes[0, 0].set_xlabel('% Vendido')
axes[0, 0].set_ylabel('Frequência')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. % Vendido por Status
vendas_status = df_imoveis.groupby('Status')['Perc_Vendido'].agg(['mean', 'std', 'count'])
vendas_status = vendas_status.sort_values('mean', ascending=False)

bars = axes[0, 1].bar(vendas_status.index, vendas_status['mean'], 
                      yerr=vendas_status['std'],
                      color=['black', 'gray', 'lightgray'],
                      alpha=0.7, edgecolor='black', linewidth=2,
                      capsize=10, error_kw={'linewidth': 2, 'ecolor': 'black'})

# Adicionar valores
for bar, mean, count in zip(bars, vendas_status['mean'], vendas_status['count']):
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                   f'{mean:.1f}%\n(n={count})',
                   ha='center', va='bottom', fontweight='bold')

axes[0, 1].set_title('% Médio Vendido por Status', fontweight='bold')
axes[0, 1].set_ylabel('% Vendido Médio')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. Performance por Incorporador
top_incorporadores = df_imoveis.groupby('Incorporador').agg({
    'Perc_Vendido': 'mean',
    'VGV_Realizado': 'sum',
    'Empreendimento': 'count'
}).sort_values('VGV_Realizado', ascending=False).head(8)

y_pos = np.arange(len(top_incorporadores))
axes[1, 0].barh(y_pos, top_incorporadores['Perc_Vendido'], 
                color='black', alpha=0.7, edgecolor='white', linewidth=1.5)

# Adicionar número de empreendimentos
for i, (idx, row) in enumerate(top_incorporadores.iterrows()):
    axes[1, 0].text(row['Perc_Vendido'] + 1, i, 
                   f"{row['Perc_Vendido']:.1f}% ({int(row['Empreendimento'])})",
                   va='center', fontsize=9)

axes[1, 0].set_yticks(y_pos)
axes[1, 0].set_yticklabels(top_incorporadores.index)
axes[1, 0].invert_yaxis()
axes[1, 0].set_xlabel('% Médio Vendido')
axes[1, 0].set_title('Performance por Incorporador (Top 8)', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. Scatter: Preço vs % Vendido
scatter = axes[1, 1].scatter(df_imoveis['Preco_m2'], df_imoveis['Perc_Vendido'],
                            s=df_imoveis['Unidades Total']*2,
                            c='black', alpha=0.5, edgecolors='white', linewidth=1)

# Linha de tendência
z = np.polyfit(df_imoveis['Preco_m2'].dropna(), df_imoveis.loc[df_imoveis['Preco_m2'].notna(), 'Perc_Vendido'], 1)
p = np.poly1d(z)
x_trend = np.linspace(df_imoveis['Preco_m2'].min(), df_imoveis['Preco_m2'].max(), 100)
axes[1, 1].plot(x_trend, p(x_trend), "--", color='gray', linewidth=2, label='Tendência')

# Calcular correlação
corr = df_imoveis[['Preco_m2', 'Perc_Vendido']].corr().iloc[0, 1]
axes[1, 1].text(0.05, 0.95, f'Correlação: {corr:.3f}', 
               transform=axes[1, 1].transAxes, 
               fontsize=11, verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='black'))

axes[1, 1].set_xlabel('Preço/m² (R$)')
axes[1, 1].set_ylabel('% Vendido')
axes[1, 1].set_title('Relação Preço/m² vs % Vendido', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Análise de performance de vendas concluída")

## 6. Segmentação de Mercado por Tipologia

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Segmentação de Mercado por Tipologia', fontsize=16, fontweight='bold', y=0.995)

# 1. Distribuição por Dormitórios
dorm_stats = df_imoveis.groupby('Dormitorios').agg({
    'Empreendimento': 'count',
    'Preco Total': 'mean',
    'Perc_Vendido': 'mean'
})

bars = axes[0, 0].bar(dorm_stats.index, dorm_stats['Empreendimento'], 
                      color='black', alpha=0.7, edgecolor='white', linewidth=2)

for bar, count in zip(bars, dorm_stats['Empreendimento']):
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height,
                   f'{int(count)}',
                   ha='center', va='bottom', fontweight='bold', fontsize=12)

axes[0, 0].set_xlabel('Número de Dormitórios')
axes[0, 0].set_ylabel('Quantidade de Unidades')
axes[0, 0].set_title('Distribuição por Dormitórios', fontweight='bold')
axes[0, 0].set_xticks(dorm_stats.index)
axes[0, 0].grid(True, alpha=0.3, axis='y')

# 2. Preço Médio por Dormitórios
axes[0, 1].plot(dorm_stats.index, dorm_stats['Preco Total']/1000, 
                marker='o', markersize=12, linewidth=3, 
                color='black', markerfacecolor='white', markeredgewidth=2)

for x, y in zip(dorm_stats.index, dorm_stats['Preco Total']/1000):
    axes[0, 1].text(x, y, f'R$ {y:.0f}k', 
                   ha='center', va='bottom', fontweight='bold', fontsize=10)

axes[0, 1].set_xlabel('Número de Dormitórios')
axes[0, 1].set_ylabel('Preço Médio (R$ mil)')
axes[0, 1].set_title('Preço Médio por Dormitórios', fontweight='bold')
axes[0, 1].set_xticks(dorm_stats.index)
axes[0, 1].grid(True, alpha=0.3)

# 3. Distribuição por Vagas
vagas_stats = df_imoveis.groupby('Vagas').size()
explode = [0.05] * len(vagas_stats)

wedges, texts, autotexts = axes[1, 0].pie(vagas_stats.values, 
                                           labels=[f'{v} vaga{"s" if v != 1 else ""}' for v in vagas_stats.index],
                                           autopct='%1.1f%%',
                                           startangle=90,
                                           colors=['black', 'darkgray', 'gray', 'lightgray'],
                                           explode=explode,
                                           wedgeprops={'edgecolor': 'white', 'linewidth': 2})

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(11)

axes[1, 0].set_title('Distribuição por Número de Vagas', fontweight='bold')

# 4. Metragem vs Preço por Dormitórios
for dorm in sorted(df_imoveis['Dormitorios'].unique()):
    data = df_imoveis[df_imoveis['Dormitorios'] == dorm]
    color_map = {1: 'lightgray', 2: 'gray', 3: 'darkgray', 4: 'black'}
    axes[1, 1].scatter(data['Metragem Privativa'], data['Preco Total']/1000,
                      label=f'{dorm} dorm',
                      alpha=0.6, s=80,
                      c=color_map.get(dorm, 'gray'),
                      edgecolors='white', linewidth=1)

axes[1, 1].set_xlabel('Metragem Privativa (m²)')
axes[1, 1].set_ylabel('Preço Total (R$ mil)')
axes[1, 1].set_title('Relação Metragem vs Preço por Tipologia', fontweight='bold')
axes[1, 1].legend(title='Dormitórios', loc='upper left')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🏠 RESUMO POR TIPOLOGIA")
print("="*80)
tipologia = df_imoveis.groupby('Dormitorios').agg({
    'Empreendimento': 'count',
    'Metragem Privativa': 'mean',
    'Preco Total': 'mean',
    'Preco_m2': 'mean',
    'Perc_Vendido': 'mean'
}).round(2)

tipologia.columns = ['Qtd', 'Metragem Média', 'Preço Médio', 'Preço/m² Médio', '% Vendido']
print(tipologia)

## 7. Análise de VGV (Valor Geral de Vendas)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análise de VGV (Valor Geral de Vendas)', fontsize=16, fontweight='bold', y=0.995)

# 1. VGV por Incorporador
vgv_incorp = df_imoveis.groupby('Incorporador').agg({
    'VGV': 'sum',
    'VGV_Realizado': 'sum'
}).sort_values('VGV', ascending=True)

y_pos = np.arange(len(vgv_incorp))
axes[0, 0].barh(y_pos, vgv_incorp['VGV']/1e6, 
                color='lightgray', alpha=0.7, label='VGV Total', edgecolor='black', linewidth=1.5)
axes[0, 0].barh(y_pos, vgv_incorp['VGV_Realizado']/1e6, 
                color='black', alpha=0.7, label='VGV Realizado', edgecolor='white', linewidth=1.5)

axes[0, 0].set_yticks(y_pos)
axes[0, 0].set_yticklabels(vgv_incorp.index)
axes[0, 0].set_xlabel('VGV (R$ Milhões)')
axes[0, 0].set_title('VGV Total vs Realizado por Incorporador', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, axis='x')

# 2. Taxa de Realização do VGV
vgv_incorp['Taxa_Realizacao'] = (vgv_incorp['VGV_Realizado'] / vgv_incorp['VGV'] * 100).sort_values(ascending=False)
vgv_sorted = vgv_incorp.sort_values('Taxa_Realizacao', ascending=False)

bars = axes[0, 1].bar(range(len(vgv_sorted)), vgv_sorted['Taxa_Realizacao'],
                      color='black', alpha=0.7, edgecolor='white', linewidth=2)

for i, (bar, taxa) in enumerate(zip(bars, vgv_sorted['Taxa_Realizacao'])):
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                   f'{taxa:.1f}%',
                   ha='center', va='bottom', fontweight='bold')

axes[0, 1].set_xticks(range(len(vgv_sorted)))
axes[0, 1].set_xticklabels(vgv_sorted.index, rotation=45, ha='right')
axes[0, 1].set_ylabel('Taxa de Realização (%)')
axes[0, 1].set_title('Taxa de Realização do VGV por Incorporador', fontweight='bold')
axes[0, 1].axhline(y=50, color='gray', linestyle='--', linewidth=2, label='50%')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3, axis='y')

# 3. VGV por Bairro
vgv_bairro = df_imoveis.groupby('Bairro')['VGV'].sum().sort_values(ascending=True)

y_pos = np.arange(len(vgv_bairro))
bars = axes[1, 0].barh(y_pos, vgv_bairro.values/1e6, 
                       color='black', alpha=0.7, edgecolor='white', linewidth=1.5)

for i, (bar, valor) in enumerate(zip(bars, vgv_bairro.values/1e6)):
    axes[1, 0].text(valor, i, f' R$ {valor:.1f}M',
                   va='center', fontweight='bold', fontsize=9)

axes[1, 0].set_yticks(y_pos)
axes[1, 0].set_yticklabels(vgv_bairro.index)
axes[1, 0].set_xlabel('VGV Total (R$ Milhões)')
axes[1, 0].set_title('VGV Total por Bairro', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# 4. Distribuição de VGV por Empreendimento
axes[1, 1].hist(df_imoveis['VGV']/1e6, bins=30, 
                color='black', alpha=0.7, edgecolor='white', linewidth=1.5)
axes[1, 1].axvline(df_imoveis['VGV'].mean()/1e6, 
                  color='gray', linestyle='--', linewidth=2, 
                  label=f"Média: R$ {df_imoveis['VGV'].mean()/1e6:.1f}M")
axes[1, 1].axvline(df_imoveis['VGV'].median()/1e6, 
                  color='darkgray', linestyle=':', linewidth=2, 
                  label=f"Mediana: R$ {df_imoveis['VGV'].median()/1e6:.1f}M")

axes[1, 1].set_xlabel('VGV (R$ Milhões)')
axes[1, 1].set_ylabel('Frequência')
axes[1, 1].set_title('Distribuição de VGV por Empreendimento', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💰 RESUMO DE VGV")
print("="*60)
print(f"VGV Total do Mercado: R$ {df_imoveis['VGV'].sum()/1e9:.2f} Bilhões")
print(f"VGV Realizado: R$ {df_imoveis['VGV_Realizado'].sum()/1e9:.2f} Bilhões")
print(f"Taxa de Realização Média: {(df_imoveis['VGV_Realizado'].sum()/df_imoveis['VGV'].sum()*100):.1f}%")
print(f"VGV Médio por Empreendimento: R$ {df_imoveis['VGV'].mean()/1e6:.2f} Milhões")

## 8. Relatório Executivo - Resumo do Mercado

In [ ]:
print("="*80)
print("📊 RELATÓRIO EXECUTIVO - ANÁLISE DO MERCADO IMOBILIÁRIO DE VITÓRIA")
print("="*80)

print("\n🏢 PANORAMA GERAL")
print("-" * 80)
print(f"Total de Empreendimentos Analisados: {len(df_imoveis):,}")
print(f"Total de Unidades no Mercado: {df_imoveis['Unidades Total'].sum():,}")
print(f"Total de Unidades Vendidas: {df_imoveis['Unidades Vendidas'].sum():,}")
print(f"Estoque Atual: {df_imoveis['Estoque Atual'].sum():,} unidades")

print("\n💰 INDICADORES DE PREÇO")
print("-" * 80)
print(f"Preço Médio por m²: R$ {df_imoveis['Preco_m2'].mean():,.2f}")
print(f"Preço Mediano por m²: R$ {df_imoveis['Preco_m2'].median():,.2f}")
print(f"Ticket Médio: R$ {df_imoveis['Preco Total'].mean():,.2f}")
print(f"Ticket Mediano: R$ {df_imoveis['Preco Total'].median():,.2f}")

print("\n📈 PERFORMANCE DE VENDAS")
print("-" * 80)
print(f"Percentual Médio Vendido: {df_imoveis['Perc_Vendido'].mean():.1f}%")
print(f"VGV Total: R$ {df_imoveis['VGV'].sum()/1e9:.2f} Bilhões")
print(f"VGV Realizado: R$ {df_imoveis['VGV_Realizado'].sum()/1e9:.2f} Bilhões")
print(f"Taxa de Realização: {(df_imoveis['VGV_Realizado'].sum()/df_imoveis['VGV'].sum()*100):.1f}%")

print("\n🏘️ TOP 3 BAIRROS MAIS CAROS (Preço/m²)")
print("-" * 80)
top_bairros_preco = df_imoveis.groupby('Bairro')['Preco_m2'].mean().sort_values(ascending=False).head(3)
for i, (bairro, preco) in enumerate(top_bairros_preco.items(), 1):
    qtd = len(df_imoveis[df_imoveis['Bairro'] == bairro])
    print(f"  {i}. {bairro}: R$ {preco:,.2f}/m² ({qtd} empreendimentos)")

print("\n🏗️ TOP 3 INCORPORADORES (Por VGV)")
print("-" * 80)
top_incorp = df_imoveis.groupby('Incorporador').agg({
    'VGV': 'sum',
    'Empreendimento': 'count',
    'Perc_Vendido': 'mean'
}).sort_values('VGV', ascending=False).head(3)

for i, (incorp, row) in enumerate(top_incorp.iterrows(), 1):
    print(f"  {i}. {incorp}:")
    print(f"     • VGV: R$ {row['VGV']/1e6:.1f} Milhões")
    print(f"     • Empreendimentos: {int(row['Empreendimento'])}")
    print(f"     • % Vendido Médio: {row['Perc_Vendido']:.1f}%")

print("\n🏠 TIPOLOGIA MAIS VENDIDA")
print("-" * 80)
tipo_popular = df_imoveis.groupby('Dormitorios').agg({
    'Empreendimento': 'count',
    'Perc_Vendido': 'mean'
}).sort_values('Empreendimento', ascending=False).head(1)

for dorm, row in tipo_popular.iterrows():
    print(f"  {int(dorm)} Dormitórios: {int(row['Empreendimento'])} unidades ({row['Perc_Vendido']:.1f}% vendido)")

print("\n" + "="*80)
print("Relatório gerado em:", pd.Timestamp.now().strftime("%d/%m/%Y %H:%M:%S"))
print("="*80)

## 9. Exportar Relatório

In [ ]:
# Criar resumo para exportação
resumo = {
    'data_analise': pd.Timestamp.now(),
    'total_empreendimentos': len(df_imoveis),
    'preco_medio_m2': df_imoveis['Preco_m2'].mean(),
    'ticket_medio': df_imoveis['Preco Total'].mean(),
    'perc_vendido_medio': df_imoveis['Perc_Vendido'].mean(),
    'vgv_total': df_imoveis['VGV'].sum(),
    'vgv_realizado': df_imoveis['VGV_Realizado'].sum()
}

# Salvar resumo
pd.DataFrame([resumo]).to_csv('../data/processed/resumo_mercado.csv', index=False)
pd.DataFrame([resumo]).to_json('../data/processed/resumo_mercado.json', orient='records', indent=2)

print("✓ Relatórios exportados:")
print("  • resumo_mercado.csv")
print("  • resumo_mercado.json")